In [1]:
import os
import math
import torch
import random
import pickle
import pandas
import calendar
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
os.environ['KMP_DUPLICATE_LIB_OK']='True'
torch.set_default_tensor_type(torch.FloatTensor)
import config

# Read and process data

In [2]:
X = torch.load('./data/pReLU_RT_ratio.data').float()
X.shape

torch.Size([9999, 8])

In [3]:
X_max = torch.max(X, dim=0)[0]
X_min = torch.min(X, dim=0)[0]

In [4]:
X_max

tensor([9.9989e+04, 1.9998e+06, 1.9998e+04, 9.9991e+04, 8.0000e-04, 1.2000e-04,
        1.9699e-01, 9.9875e+00])

In [5]:
X_min

tensor([1.0000e+04, 5.0000e+05, 1.0000e+03, 3.0000e+04, 2.0000e-04, 8.0000e-05,
        5.1912e-03, 1.6667e+00])

In [6]:
Xn = config.Normalization(X, X_min, X_max)
Xn

tensor([[0.7501, 0.2500, 0.2500,  ..., 0.7500, 0.4347, 0.5099],
        [0.2500, 0.7501, 0.7501,  ..., 0.2500, 0.0772, 0.2671],
        [0.3750, 0.3750, 0.6251,  ..., 0.1250, 0.1876, 0.4006],
        ...,
        [0.5671, 0.4215, 0.4828,  ..., 0.6250, 0.2540, 0.5254],
        [0.0671, 0.9216, 0.9828,  ..., 0.1250, 0.0174, 0.2719],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0772, 0.1002]])

In [7]:
Xn.max(0), Xn.min(0)

(torch.return_types.max(
 values=tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
 indices=tensor([5459, 4367, 4175, 5439, 1573,  101,  168, 8989])),
 torch.return_types.min(
 values=tensor([0., 0., 0., 0., 0., 0., 0., 0.]),
 indices=tensor([9998, 9998, 9998, 9998, 1489,   74,  253, 7679])))

In [8]:
Y = torch.load('./data/pReLU_power.data').view(-1, 1).float()
Y.shape

torch.Size([9999, 1])

In [9]:
Y_max = torch.max(Y, dim=0)[0]
Y_min = torch.min(Y, dim=0)[0]

In [10]:
Y_min

tensor([1.7716e-05])

In [11]:
Y_max

tensor([8.2733e-05])

In [12]:
Yn = config.Normalization(Y, Y_min, Y_max)

In [13]:
Yn.max(), Yn.min()

(tensor(1.), tensor(0.))

# Make dataset

In [14]:
E_train = int(X.shape[0]*0.7)
E_valid = int(X.shape[0]*0.2)
E_test  = int(X.shape[0] - E_train - E_valid)

config.SetSeed()
index = torch.randperm(X.shape[0])
index_learn = index[:E_train+E_valid]
index_train = index[:E_train]
index_valid = index[E_train:E_train+E_valid]
index_test  = index[-E_test:]

X_learn, Y_learn = X[index_learn,:], Y[index_learn,:]
X_train, Y_train = X[index_train,:], Y[index_train,:]
X_valid, Y_valid = X[index_valid,:], Y[index_valid,:]
X_test,  Y_test  = X[index_test,:] , Y[index_test,:]

Xn_learn, Yn_learn = Xn[index_learn,:], Yn[index_learn,:]
Xn_train, Yn_train = Xn[index_train,:], Yn[index_train,:]
Xn_valid, Yn_valid = Xn[index_valid,:], Yn[index_valid,:]
Xn_test,  Yn_test  = Xn[index_test,:] , Yn[index_test,:]

In [15]:
a = {'Xn': Xn, 'Yn': Yn, 
     'X': X,   'Y': Y, 
     
     'Xn_learn': Xn_learn, 'Yn_learn': Yn_learn, 
     'Xn_train': Xn_train, 'Yn_train': Yn_train, 
     'Xn_valid': Xn_valid, 'Yn_valid': Yn_valid, 
     'Xn_test' : Xn_test,  'Yn_test' : Yn_test,
     
     'X_learn': X_learn, 'Y_learn': Y_learn, 
     'X_train': X_train, 'Y_train': Y_train, 
     'X_valid': X_valid, 'Y_valid': Y_valid, 
     'X_test' : X_test,  'Y_test' : Y_test,
     
     'X_max':X_max, 'X_min':X_min,
     'Y_max':Y_max, 'Y_min':Y_min,}

torch.save(a, './data/pReLU_power.ds')